In [42]:
from z3 import * 

In [43]:
# Da prendere
def toBinary(num, length = None):
    num_bin = bin(num).split("b")[-1]
    if length:
        return "0"*(length - len(num_bin)) + num_bin
    return num_bin

def at_least_one_bw(bool_vars):
    return Or(bool_vars)

def at_most_one_bw(bool_vars, name):
    constraints = []
    n = len(bool_vars)
    m = math.ceil(math.log2(n))
    r = [Bool(f"r_{name}_{i}") for i in range(m)]
    binaries = [toBinary(i, m) for i in range(n)]
    for i in range(n):
        for j in range(m):
            phi = Not(r[j])
            if binaries[i][j] == "1":
                phi = r[j]
            constraints.append(Or(Not(bool_vars[i]), phi))
    return And(constraints)

def exactly_one_bw(bool_vars, name):
    return And(at_least_one_bw(bool_vars), at_most_one_bw(bool_vars, name))

def my_max(vs):
    m = vs[0]
    for v in vs[1:]:
        m = If(v > m, v, m)
    return m

def is_circuit_element(i, val, iter, vector, items):    
    if iter == items:
        return False
    return And(
        And(
        Not(vector[i] == items), Or(vector[i]==val, 
                                        is_circuit_element(vector[i], val, iter+1, vector, items))))


In [44]:
# # Create a solver
# s = Solver()

# # Utils function
# def is_circuit_element(i, val, iter, vector, items):
    
#     if iter == items:
#         return False
#     return And(
#         And(
#         Not(vector[i] == items), Or(vector[i]==val, 
#                                         is_circuit_element(vector[i], val, iter+1, vector, items))))


# vector = Array('x', IntSort(), IntSort())
# print(type(vector))

# s.add(vector[9] == 0)
# s.add(vector[0] == 1)
# s.add(vector[1] == 2)
# s.add(vector[2] == 3)
# s.add(vector[3] == 4)
# s.add(vector[4] == 5)
# s.add(vector[5] == 7)
# s.add(vector[6] == -1)
# s.add(vector[7] == 8)
# s.add(vector[8] == 9)

# for i in range(10):
#     s.add(Implies(
#         And(Select(vector, i) != -1, Select(vector, i) != 9)  , 
#         is_circuit_element(9, vector[i], 0, vector, 9)
#     )
#     )

# s.add(vector[9] != 9)


# for i in range(10):
#     s.add(Select(vector, i) >= -1)
#     s.add(Select(vector, i) <= 9)
#     s.add(
#         Implies(Select(vector, i) != -1, Select(vector, vector[i]) != -1
#     )
#     )
# for j in range(10):
#     s.add(vector[j] != j)
    
# for j in range(0, 10):
#     s.add(at_most_one_bw([Select(vector, i) == j for i in range(0, 10)], f"{j}"))
    
# s.add(
#     Or([Select(vector, i) == 9 for i in range(10)])
# )

# # for i in range(10):
# #     s.add(exactly_one_bw([vector[j] == i for j in range(10)], f'{i}'))

# s.check()


# m = s.model()

# print([m.evaluate(vector[i]) for i in range(10)])

In [45]:
# Smt model 1
# Intsance
couriers = 3
items = 7
couriers_size = [15,10,7]
items_size = [3,2,6,8,5,4,4]
distances = [
    [0,3,3,6,5,6,6,2],
    [3,0,4,3,4,7,7,3],
    [3,4,0,7,6,3,5,3],
    [6,3,7,0,3,6,6,4],
    [5,4,6,3,0,3,3,3],
    [6,7,3,6,3,0,2,4],
    [6,7,5,6,3,2,0,4],
    [2,3,3,4,3,4,4,0]]



In [46]:
# variables
import time
couriers_loads = [Int(f'loads{i}') for i in range(couriers)]

final_distances = [Int(f"dist{i}") for i in range(couriers)]

asg = [Array(f"asg{i}", IntSort(), IntSort()) for i in range(couriers)]

# constraints
s = Optimize()
start_time = time.time()

array_of_distances = [Array(f'distances{i}', IntSort(), IntSort()) for i in range(items+1)]

#define the distances
for i in range(items + 1):
    for j in range(items + 1):
        s.add(array_of_distances[i][j] == distances[i][j])

# Compreso nel **
#for k in range(couriers):
#    s.add(asg[k][items] != items)

for k in range(couriers):
    for i in range(items+1):
        s.add(Implies(
                 And(asg[k][i] != -1, asg[k][i] != items), 
                 is_circuit_element(items, asg[k][i], 0, asg[k], items+1)
             )
        )

for k in range(couriers):
    for i in range(items+1):
        s.add(asg[k][i] >= -1)
        s.add(asg[k][i] <= items)
        s.add(
            Implies(asg[k][i] != -1, asg[k][asg[k][i]] != -1)
        )
# **
for k in range(couriers):
    for j in range(items+1):
        s.add(asg[k][j] != j)

# Rivedere
for k in range(couriers):
    for j in range(items+1):
        s.add(at_most_one_bw([asg[k][i] == j for i in range(items+1)], f"A{j}{k}"))

for k in range(couriers):
    s.add(
        Or([asg[k][i] == items for i in range(items+1)])
    )

for i in range(items):
    s.add(exactly_one_bw(
        [asg[k][j] == i for k in range(couriers) for j in range(items+1)], f'{i}')
    )

for k in range(couriers):
    s.add(couriers_loads[k] == Sum(
        [If(asg[k][i] != -1, items_size[i], 0) for i in range(items)]))
    

for k in range(couriers):
    s.add(
        couriers_loads[k] <= couriers_size[k]
    )

for k in range(couriers):
    s.add(final_distances[k] == Sum(
        [If(asg[k][i] != -1, array_of_distances[i][asg[k][i]], 0) for i in range(items+1)]))
    

max = Int(f"max")

s.add(max == my_max([final_distances[i] for i in range(couriers)]))

s.minimize(max)
print("Time = ", round(time.time() - start_time,2))
print(s.check())

m = s.model()

print([[m.evaluate(asg[k][j]) for j in range(items+1)] for k in range(couriers)])


print([m.evaluate(final_distances[i]) for i in range(couriers)])

Time =  0.48
sat
[[-1, -1, 5, -1, 7, 4, -1, 2], [-1, 3, -1, 7, -1, -1, -1, 1], [7, -1, -1, -1, -1, -1, 0, 6]]
[12, 10, 12]


In [47]:
vector1 = Array('x', IntSort(), IntSort())
vector2 = Array('x', IntSort(), IntSort())
s.add(vector1[0] == 5)
s.add(vector1[1] == 2)
s.add(vector1[2] == 3)
s.add(vector1[3] == 0)
s.add(vector1[4] == 0)
s.add(vector1[5] == 0)

s.add(vector2[0] == 2)
s.add(vector2[1] == 3)
s.add(vector2[2] == 0)
s.add(vector2[3] == 5)
s.add(vector2[4] == 4)
s.add(vector2[5] == 0)

s.add(vector2[0] != 5)
s.add(vector1[0] == 5)


for i in range(6):
    s.add(Implies(vector1[i] == 0, vector2[i] == 0))
    s.add(Implies(vector1[i] != 0, vector2[i] != 0))


s.add(exactly_one_bw([vector2[i] == 5 for i in range(6)], f"A{i}"))

# Each arrival point must be the next starting one if different from the origin
for i in range(6):
    s.add(Implies(vector2[i] != 5,vector1[i+1] == vector2[i]))





s.check()

unsat